# <font color=blue>Project Objective<font>

1. To consolidate the event data CSVs into a single CSV data file
2. To create 3 Apache Cassandra tables modelled on the below 3 queries:
        a. To find the artist, song title and song's length in the music app history that was heard during 
           sessionId = 338 and itemInSession = 4 
        b. To find the name of artist, song (sorted by itemInSession) and user (first and last name) for 
           userid= 10, sessionid = 182
        c. To find every user name (first and last) in my music app history who listened to the song 
           'All Hands Against His Own'
3. To load the tables from the single CSV file created

#### <font color=blue>Importing Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### <font color=blue>Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### <font color=blue>Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# reading csv file for every filepath in the file path list and creating a csv header
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## <font color=red>event_datafile_new.csv</font> <font color=blue>is located within the Workspace directory & contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### <font color=blue>Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### <font color=blue>Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### <font color=blue>Connecting to the Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### <font color=blue> Data Modelling based on the queries

### <font color=red>Query 1<font>
#### To find the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### <font color=blue>Creating Table

In [8]:
query = "DROP TABLE IF EXISTS song_details_by_sessioninfo"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Primary Key: sessionId(Partition Key) + itemInSession(Clustering Column)
# sessionId - (i) unique column used for uniform distribution of data 
#            (ii) used in Query 1's where clause as 1st condition
# itemInSession - (i) Data to be sorted within partition by itemInSession 
#                 (ii) used in Query 1's where clause as 2nd condition
query = "CREATE TABLE IF NOT EXISTS song_details_by_sessioninfo "
query = query + "(artist TEXT, song TEXT, length DECIMAL, sessionId INT, itemInSession INT, \
PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### <font color=blue>Loading data into the Table from CSV file

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO song_details_by_sessioninfo (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### <font color=blue>Data Verification

In [10]:
query = "SELECT artist, song, length from song_details_by_sessioninfo "
query = query + (" WHERE sessionId = 338 AND itemInSession = 4") 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(rows._current_rows)
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### <font color=red>Query 2<font>
#### To find only the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### <font color=blue>Creating table<font>

In [11]:
query = "DROP TABLE IF EXISTS songinfo_by_userinfo"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


# Primary Key: (userId + sessionId)(Partition Key) + itemInSession(Clustering Column)
# sessionId + Session - (i) together are unique and will ensure uniform distribution of data 
#                       (ii) both are used in Query 2's where clause in the same order
# itemInSession - (i) Data to be sorted within partition by itemInSession in ascending order
query = "CREATE TABLE IF NOT EXISTS songinfo_by_userinfo "
query = query + "(userid INT, sessionid INT, itemInSession INT, artist TEXT, song TEXT, \
firstName TEXT, lastName TEXT, \
PRIMARY KEY ((userid, sessionid), itemInSession)) \
WITH CLUSTERING ORDER BY (itemInSession ASC)"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### <font color=blue>Loading data into the Table from the CSV file

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_userinfo (userid, sessionid, itemInSession, artist, \
        song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### <font color=blue>Data Verification

In [13]:
query = "SELECT artist, song, firstName, lastName from songinfo_by_userinfo "
query = query + (" WHERE userId = 10 AND sessionid = 182 ORDER BY itemInSession ASC") 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(rows._current_rows)
df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### <font color=red>Query 3<font>
#### To find every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Creating table

In [14]:
query = "DROP TABLE IF EXISTS userinfo_by_songlistened"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


# Primary Key: song + userId(Partition Key)
# Song + UserId will make primary key unique and there will be uniform distribution of data 


query = "CREATE TABLE IF NOT EXISTS userinfo_by_songlistened "
query = query + "(userid INT, song TEXT, firstName TEXT, lastName TEXT, \
        PRIMARY KEY (song, userid))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### <font color=blue>Loading data into the the table from the CSV file

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = "INSERT INTO userinfo_by_songlistened (userid, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

#### <font color=blue>Data Verification

In [16]:
query = "SELECT firstName, lastName from userinfo_by_songlistened "
query = query + (" WHERE song = 'All Hands Against His Own'") 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(rows._current_rows)
df                   

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### <font color=blue>Dropping the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS song_details_by_sessioninfo"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "DROP TABLE IF EXISTS songinfo_by_userinfo"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

query = "DROP TABLE IF EXISTS userinfo_by_songlistened"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### <font color=blue>Closing the session and cluster connections

In [18]:
session.shutdown()
cluster.shutdown()